In [1]:
import pandas as pd
import numpy as np
import math
import time
from tqdm import tqdm

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import lightgbm as lgb
from sklearn.metrics import (roc_curve, auc, roc_auc_score,
                             confusion_matrix)
import matplotlib.pyplot as plt
import itertools


In [2]:
# read BPIC12.csv and Add label
# label = {'Activity' : 'O_ACCEPTED-COMPLETE'}

df = pd.DataFrame()

def label(self):
    df = pd.read_csv('BPIC12.csv')
    df = df[['Case ID', 'Activity', 'Resource', 'Complete Timestamp']]
    df['label'] = ""
    df.loc[df['Activity'] == 'O_ACCEPTED-COMPLETE', 'label'] = '1'
    df.loc[df['Activity'] != 'O_ACCEPTED-COMPLETE', 'label'] = '0'
    return df

df = label(df)

In [3]:
# error check
count = len(df[df['label'] == '1'])
print(count)

2243


In [4]:
# dict : drop_act, bucketing, encoding, model

# drop_act = {2, 4, 6, 8}
# bucketing = {1, 2*mean_trace_length}
# encoding = {'aggregate', 'index'}
# model = {'DT', 'RF', 'XGB', 'LGBM'}

parameter = {'drop_act':[2,4,6,8], 
    'bucketing':[1, 'mean_trace_length'],
    'encoding':['aggregate', 'index'],
    'model':['DT', 'RF', 'XGB', 'LGBM']}

     # need to modify bucketing

In [25]:
def random_machine():
    i = np.random.randint(0, 4)
    j = np.random.randint(0, 2)
    k = np.random.randint(0, 2)
    v = np.random.randint(0, 4)

    tuple = [parameter['drop_act'][i],
            parameter['bucketing'][j],
            parameter['encoding'][k],
            parameter['model'][v]]

    return tuple

print(random_machine())

[6, 1, 'index', 'RF']


In [51]:
result_random_ML = []

for _ in range(100):
    rand = random_machine()
    result_random_ML.append(rand)

In [49]:
# class initial_population():

#     def __init__(self):
#         pass

#     def initial_pop():
#         result_random_ML = []

#         for _ in range(100):
#             rand = random_machine()
#             result_random_ML.append(rand)

#         return result_random_ML

In [50]:
initial_population.initial_pop()

[[6, 'mean_trace_length', 'aggregate', 'DT'],
 [4, 1, 'index', 'LGBM'],
 [8, 1, 'index', 'XGB'],
 [2, 'mean_trace_length', 'index', 'RF'],
 [2, 'mean_trace_length', 'index', 'RF'],
 [6, 'mean_trace_length', 'aggregate', 'XGB'],
 [8, 'mean_trace_length', 'index', 'RF'],
 [4, 1, 'aggregate', 'RF'],
 [2, 'mean_trace_length', 'aggregate', 'LGBM'],
 [4, 'mean_trace_length', 'index', 'XGB'],
 [8, 1, 'index', 'XGB'],
 [2, 1, 'index', 'XGB'],
 [8, 1, 'aggregate', 'RF'],
 [4, 'mean_trace_length', 'index', 'LGBM'],
 [6, 1, 'aggregate', 'DT'],
 [8, 1, 'index', 'LGBM'],
 [6, 1, 'aggregate', 'LGBM'],
 [8, 1, 'aggregate', 'RF'],
 [2, 1, 'aggregate', 'XGB'],
 [2, 'mean_trace_length', 'index', 'XGB'],
 [6, 1, 'aggregate', 'XGB'],
 [4, 'mean_trace_length', 'aggregate', 'LGBM'],
 [6, 'mean_trace_length', 'index', 'LGBM'],
 [8, 1, 'aggregate', 'RF'],
 [2, 'mean_trace_length', 'aggregate', 'DT'],
 [4, 'mean_trace_length', 'aggregate', 'LGBM'],
 [4, 1, 'index', 'DT'],
 [4, 'mean_trace_length', 'aggregate',

In [52]:
# class drop_activity(initial_population):

drop_df = pd.DataFrame()
    
drop_list = [i[0] for i in result_random_ML]
print(drop_list)



      
# mask = df['num_caseid'] <= 2
# df = df.drop(df[mask].index)

[2, 4, 4, 6, 6, 2, 8, 6, 2, 8, 4, 8, 2, 4, 6, 4, 2, 8, 6, 8, 4, 8, 6, 8, 2, 6, 4, 8, 6, 6, 8, 8, 2, 8, 8, 4, 6, 6, 4, 2, 6, 6, 8, 8, 8, 2, 2, 2, 8, 6, 8, 4, 4, 4, 6, 2, 6, 2, 6, 8, 8, 8, 6, 6, 8, 8, 8, 6, 4, 4, 8, 8, 4, 8, 6, 2, 6, 8, 4, 6, 2, 6, 6, 4, 8, 2, 8, 6, 8, 8, 2, 4, 6, 4, 6, 6, 8, 6, 4, 2]


In [64]:
mask = df['num_caseid'] <= drop_list[i]
df_drop = df.drop(df[mask].index)
    
df_drop

# if drop_list[0] == 2:
    

,Case ID,Activity,Resource,Complete Timestamp,label,num_caseid
0,173688,A_SUBMITTED-COMPLETE,112.0,2011/10/01 07:38:44.546,0,26.0
1,173688,A_PARTLYSUBMITTED-COMPLETE,112.0,2011/10/01 07:38:44.880,0,26.0
2,173688,A_PREACCEPTED-COMPLETE,112.0,2011/10/01 07:39:37.906,0,26.0
3,173688,W_Completeren aanvraag-SCHEDULE,112.0,2011/10/01 07:39:38.875,0,26.0
4,173688,W_Completeren aanvraag-START,NaN,2011/10/01 18:36:46.437,0,26.0
...,...,...,...,...,...,...
262195,214376,A_PARTLYSUBMITTED-COMPLETE,112.0,2012/03/01 07:51:17.423,0,6.0
262196,214376,W_Afhandelen leads-SCHEDULE,112.0,2012/03/01 07:52:01.287,0,6.0
262197,214376,W_Afhandelen leads-START,11169.0,2012/03/01 17:26:46.736,0,6.0
262198,214376,A_DECLINED-COMPLETE,11169.0,2012/03/01 17:27:37.118,0,6.0


In [12]:
# drop_act 함수 구현하기

# num_rows = (df['Case ID'] == 173688).sum()
# print(num_rows)

# df.loc[df['Case ID'] == 173688, 'num_caseid'] = (df['Case ID'] == 173688).sum()
# df

df_copy = df # 원본 저장용

def drop_act():
    for i in tqdm(range(len(df['Case ID']))):
        df.loc[df['Case ID'] == i, 'num_caseid'] = (df['Case ID'] == i).sum()
    return df


In [15]:
drop_act()

100%|██████████| 262200/262200 [04:37<00:00, 944.19it/s]


,Case ID,Activity,Resource,Complete Timestamp,label,num_caseid
0,173688,A_SUBMITTED-COMPLETE,112.0,2011/10/01 07:38:44.546,0,26.0
1,173688,A_PARTLYSUBMITTED-COMPLETE,112.0,2011/10/01 07:38:44.880,0,26.0
2,173688,A_PREACCEPTED-COMPLETE,112.0,2011/10/01 07:39:37.906,0,26.0
3,173688,W_Completeren aanvraag-SCHEDULE,112.0,2011/10/01 07:39:38.875,0,26.0
4,173688,W_Completeren aanvraag-START,NaN,2011/10/01 18:36:46.437,0,26.0
...,...,...,...,...,...,...
262195,214376,A_PARTLYSUBMITTED-COMPLETE,112.0,2012/03/01 07:51:17.423,0,6.0
262196,214376,W_Afhandelen leads-SCHEDULE,112.0,2012/03/01 07:52:01.287,0,6.0
262197,214376,W_Afhandelen leads-START,11169.0,2012/03/01 17:26:46.736,0,6.0
262198,214376,A_DECLINED-COMPLETE,11169.0,2012/03/01 17:27:37.118,0,6.0


In [9]:
# if 'drop_act':[2,4,6,8] in parameter is 2,
# delete df['num_caseid'] =< 2

if 
    mask = df['num_caseid'] <= 2
    df = df.drop(df[mask].index)


SyntaxError: expected ':' (477507703.py, line 4)

In [ ]:
# need to modify, error function
# AUC
def get_auc_scores(random_ML, X_train, X_test, y_train, y_test):
   '''Prints the AUC scores for training and testing data
   and returns testing score'''
 
 y_train_score = clf.predict_proba(X_train)[:, 1]
 y_test_score = clf.predict_proba(X_test)[:, 1]
 auc_train = roc_auc_score(y_train, y_train_score)
 auc_test = roc_auc_score(y_test, y_test_score)
 print(f'''
    Training AUC: {auc_train}
    Testing AUC: {auc_test}''')
 
 return y_test_score


In [16]:
#sc
def score():
    pass

#re
def reliability():
    pass

#te
def time_efficiency():
    time = end - start
    #te = (max(time) - time / max(time) - min(time))
    return time

#se
def score_efficiency():
    pass

In [17]:
# class genetic_algorithm()

In [18]:
# Data pre-processing

X = df.drop('label', axis=1)
y = df['label']

In [19]:
# one-hot encoding
# train and test set for decision tree
def train_set(self):
    df_train = pd.get_dummies(data = X, columns = ['Activity'], prefix = 'Activity')
    df_train['Complete Timestamp'] = pd.to_datetime(df_train['Complete Timestamp'], format='%Y/%m/%d %H:%M:%S.%f')

    # datetime to float for DT
    df_train['Complete Timestamp'] = (df_train['Complete Timestamp'] - df_train['Complete Timestamp'].min()) / np.timedelta64(1,'D')
    df_train

    # remove NaN for DT
    df_train = df_train.fillna(0)
    return df_train

a = train_set(X)
df_train = pd.DataFrame(a)

In [20]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size=0.2, random_state=2023)

In [21]:
# machine learning
# decision tree
def DT():
    clf_dt = tree.DecisionTreeClassifier()
    clf_dt = clf_dt.fit(X_train, y_train)

    # Make predictions on the test data
    predictions = clf_dt.predict(X_test)

    # Evaluate the model's performance
    accuracy = clf_dt.score(X_test, y_test)
    print('Accuracy:', accuracy)

# random forest
def RF(): # kernel crush was that the RAM is too small.
    # Create a Random Forest classifier
    clf_rf = RandomForestClassifier()

    # Train the classifier on the training data
    clf_rf.fit(X_train, y_train)

    # Make predictions on the test data
    predictions = clf_rf.predict(X_test)

    # Evaluate the model's performance
    accuracy = clf_rf.score(X_test, y_test)
    print('Accuracy:', accuracy)

# XGB
# Encode the target variable
    
le = LabelEncoder()
y = le.fit_transform(y)

def XGB():
    xgb = XGBClassifier(n_estimators=500, learning_rate=0.1, max_depth=4)
    X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size=0.2, random_state=2023)
    xgb.fit(X_train, y_train)
    xgb_pred = xgb.predict(X_test)

    accuracy = accuracy_score(y_test, xgb_pred)
    precision = precision_score(y_test, xgb_pred)
    recall = recall_score(y_test, xgb_pred)
    f1 = f1_score(y_test, xgb_pred)

    print('Accuracy:', accuracy)
    print('Precision:', precision)
    print('Recall:', recall)
    print('F1 Score:', f1)

# LGBM

def LGBM():
    lgb_clf = lgb.LGBMClassifier(num_leaves=31, objective='binary')
    lgb_clf.fit(X_train, y_train)
    y_pred = lgb_clf.predict(X_test)
    acc_lgbm = accuracy_score(y_test, y_pred)
    print('Accuracy', acc_lgbm)